<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/master/Demos/Ch6_RepeatedSolves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why learn about LU? Why not just use `solve` (scipy) or backslash (Matlab)?

It's often quite handy. For example, if you have the same matrix $A$ but many different right-hand-sides $b$, then you only need to find the LU decomposition of $A$ once (this is the expensive, $O(n^3)$ part), and then for each right-hand-side, just do backsubstitution (relatively cheap, at $O(n^2)$).

For Matlab, the equivalent of `scipy.linalg.solve` is 
backslash `\`, aka [`mldivide`](https://www.mathworks.com/help/matlab/ref/mldivide.html).  See more at the [Matlab systems-of-equations](https://www.mathworks.com/help/matlab/math/systems-of-linear-equations.html) documentation which also shows you how to use the new [`decomposition`](https://www.mathworks.com/help/matlab/ref/decomposition.html) function ).  You can either ask for a LU factorization explicitly and save it, or as for a vague "decomposition" and Matlab will guess a factorization (LU, Cholesky, etc) and then save it, and then you can use this with the backslash `\` seemlessly.  Neat!

*To be more accurate, the true equivalent of `scipy.linalg.solve` is Matlab's [`linsolve`](https://www.mathworks.com/help/matlab/ref/linsolve.html).  It's like backslash but it doesn't guess the structure of the matrix for you, rather you have to tell it. So that can be more annoying, but it can also save computational time (if you know it's a symmetric matrix, no need for Matlab to numerically check if it is, etc.)


### Below is a timing example about the benefits of precomputing an LU factorization

In [13]:
import numpy as np
from numpy.random import default_rng
rng = default_rng(123)
from timeit import timeit, time

from scipy.linalg import solve, lu_factor, lu_solve

In [9]:
n = int(3e3)
A = rng.standard_normal((n,n))
nTrials = 10

For each new right-hand-side, just call `solve`. This is wasteful, since $A$ isn't changing

In [11]:
%%time
for rep in range(nTrials):
  b = rng.standard_normal(n)
  x = solve(A,b)

CPU times: user 12.8 s, sys: 1.42 s, total: 14.2 s
Wall time: 7.26 s


This time, let's precompute the LU factorization of $A$ and re-use it. Much faster!

In [15]:
%%time
LU, piv = lu_factor(A)
for rep in range(nTrials):
  b = rng.standard_normal(n)
  x = lu_solve((LU,piv),b)

CPU times: user 1.22 s, sys: 80.9 ms, total: 1.3 s
Wall time: 682 ms
